In [2]:
import boto3
import json
import datetime
from IPython.display import display, JSON

bedrock_agent = boto3.client("bedrock-agent")
bedrock_runtime = boto3.client("bedrock-runtime")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")

model_id = "amazon.nova-micro-v1:0"

In [3]:
prompt_name = f"job-description-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

template_text = '''
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words
'''
response = bedrock_agent.create_prompt(
    name=prompt_name,
    description="Generates inclusive job descriptions from structured inputs",
    defaultVariant="v1",
    variants=[
        {
            "name": "v1",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.7,
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        }
    ]
)
print("\n==================== Response Object ====================\n")
display(JSON(response))

print("\n==================== Prompt ARN ====================\n")
print(response['arn'])


==================== Response Object ====================



<IPython.core.display.JSON object>


==================== Prompt ARN ====================

arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC


In [4]:
response = bedrock_agent.create_prompt_version(
    description='Initial prompt for creating job description documents.',
    promptIdentifier="arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC"
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC:1


In [5]:
prompt_arn_with_version = "arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn_with_version,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\n==================== Response Text ====================\n")
print(response['output']['message']['content'][0]['text'])


==================== Response Text ====================

**Job Title: UX Designer**

**Summary:**
We are seeking a skilled and innovative UX Designer to join our dynamic team. The ideal candidate will play a crucial role in designing user interfaces, conducting usability testing, and collaborating with cross-functional product teams to deliver exceptional user experiences.

**Responsibilities:**
- Design intuitive and user-friendly interfaces.
- Conduct comprehensive usability testing and analyze results to inform design decisions.
- Collaborate closely with product managers, developers, and other stakeholders to ensure alignment and project success.

**Requirements:**
- Minimum of 3+ years of experience in UX design.
- Proficiency with Figma for UI design.
- Strong knowledge of HTML/CSS.
- Excellent communication skills to effectively convey ideas and collaborate with team members.
- Flexibility to work in either New York or remotely.

**Location:** New York or remote

**Work Type:**

In [6]:
def handle_response_stream(response):
    try:
        event_stream = response['optimizedPrompt']
        for event in event_stream:
            if 'optimizedPromptEvent' in event:
                print("\n==================== OPTIMIZED PROMPT ====================\n")
                print(event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text'])
    except Exception as e:
        raise e


prompt_input = {
    "textPrompt": {
        "text": template_text
    }
}

response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

print("\n==================== ORIGINAL PROMPT ====================\n")
print(template_text)
handle_response_stream(response)


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== OPTIMIZED PROMPT ====================

"## Role\nYou are an expert HR assistant specializing in crafting compelling, inclusive job descriptions that attract diverse talent.\n\n## Task\nCreate a professional job description that is clear, engaging, and accessible to all candidates.\n\n## Input Information\n\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Instructions\n\nFollow these guidelines to create an effective job description:\n\n1. **Op

In [7]:
prompt_identifier = "arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC"

existing_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print("\n==================== ORIGINAL PROMPT ====================\n")
print(existing_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


updated_variants = existing_prompt["variants"]
for variant in updated_variants:
    if variant["templateType"] == "TEXT":
        variant["templateConfiguration"]["text"]["text"] = "## Role\nYou are an expert HR assistant specializing in crafting compelling, inclusive job descriptions that attract diverse talent.\n\n## Task\nCreate a professional job description that is clear, engaging, and accessible to all candidates.\n\n## Input Information\n\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Instructions\n\nFollow these guidelines to create an effective job description:\n\n1. **Opening Summary**: Begin with a compelling 2-3 sentence summary that captures the essence of the role and its impact on the organization.\n\n2. **Structure**: Organize the job description into clear sections:\n   - Role Overview\n   - Key Responsibilities (use bullet points for clarity)\n   - Required Qualifications\n   - Location and Work Arrangement\n\n3. **Inclusive Language**: \n   - Use gender-neutral terms and avoid gendered pronouns\n   - Focus on essential qualifications only\n   - Avoid jargon or unnecessarily complex terminology\n   - Use welcoming, accessible language that encourages diverse applicants\n\n4. **Conciseness**: Keep the entire description under 250 words while maintaining clarity and completeness.\n\n5. **Tone**: Maintain a professional yet approachable tone that reflects a welcoming workplace culture.\n\n6. **Formatting**: Use bullet points, short paragraphs, and clear headings to enhance readability.\n\nProvide the complete job description immediately without any preamble, explanations, or meta-commentary."


response = bedrock_agent.update_prompt(
    promptIdentifier=prompt_identifier,
    name=existing_prompt["name"],
    description=existing_prompt["description"],
    defaultVariant=existing_prompt["defaultVariant"],
    variants=updated_variants
)

print("\n==================== UPDATED PROMPT ====================\n")

updated_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print(updated_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== UPDATED PROMPT ====================

## Role
You are an expert HR assistant specializing in crafting compelling, inclusive job descriptions that attract diverse talent.

## Task
Create a professional job description that is clear, engaging, and accessible to all candidates.

## Input Information

### Job Title
{{job_title}}

### Responsibilities
{{responsibilities}}

### Requirements
{{requirements}}

### Location
{{location}}

### Work Type
{{work_type}}

## Instructions

Follow these guidelines to create an effective job description:

1. **Opening Summary**: Begin with a 

In [8]:
response = bedrock_agent.create_prompt_version(
    description='Optimized prompt for creating job description documents.',
    promptIdentifier=prompt_identifier
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC:2


In [9]:
prompt_arn = "arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**Summary:**
We are seeking a creative and inclusive UX Designer to join our dynamic team in New York or remotely. The ideal candidate will bring a blend of technical expertise and empathetic design skills to craft intuitive user interfaces and lead usability testing initiatives. You will collaborate closely with cross-functional product teams to ensure our products meet user needs and exceed expectations.

**Responsibilities:**
- Design compelling and accessible user interfaces
- Conduct usability testing and analyze user feedback to refine designs
- Collaborate with product managers, developers, and other stakeholders to deliver outstanding user experiences

**Requirements:**
- 3+ years of experience in UX design
- Proficiency in Figma for design prototyping
- Knowledge of HTML/CSS to implement design specifications
- Strong verbal and written communication skills
- Ability to work inclusively wit

In [10]:
prompt_arn = "arn:aws:bedrock:us-east-1:257317331274:prompt/2EUF15HENC:2"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**Role Overview**
Join our innovative team as a UX Designer and shape the digital experiences that will define the future of our organization. This role is pivotal in crafting seamless, user-friendly interfaces that resonate with our diverse user base.

**Key Responsibilities**
- Design intuitive user interfaces that enhance user engagement and satisfaction
- Conduct thorough usability testing to identify and rectify user experience bottlenecks
- Collaborate closely with product teams to align designs with project goals and user needs

**Required Qualifications**
- 3+ years of experience in UX design
- Proficiency in Figma for design and prototyping
- Familiarity with HTML/CSS to implement design specifications
- Strong communication skills to effectively convey ideas and feedback

**Location and Work Arrangement**
- Based in New York or eligible for remote work
- Full-time position

We value divers

In [11]:
response = bedrock_agent.list_prompts()

for prompt in response['promptSummaries']:
    prompt_id = prompt['id']  # The correct key is 'id', not 'promptId'
    print(f"Deleting prompt: {prompt['name']} (ID: {prompt_id})")
    bedrock_agent.delete_prompt(promptIdentifier=prompt_id)

print("All prompts deleted successfully")

Deleting prompt: job-description-20260222101448 (ID: 2EUF15HENC)
All prompts deleted successfully
